# ECO 모델의 추론

ECO 모델을 구현하고, 동영상 데이터의 클래스 분류를 수행합니다


# 9.5 학습 목표

1.	ECO 모델을 구현할 수 있다
2.	학습된 ECO 모델을 자신의 모델에 로드할 수 있다
3.	ECO 모델을 사용하여, 테스트 데이터를 추론할 수 있다



# 사전 준비

- "weights" 폴더에 다운로드한 "ECO_Lite_rgb_model_Kinetics.pth.tar"를 배치하세요.

https://github.com/mzolfaghari/ECO-pytorch 의

https://drive.google.com/open?id=1XNIq7byciKgrn011jLBggd2g79jKX4uD


- 9.2부터 9.4절까지 구현한 내용을 "utils"에 준비해놓고 있습니다.

이를 이용합니다.

In [1]:
import os

import torch
import torch.nn as nn
from torch.nn import init

In [2]:
# weights 폴더가 없으면 작성한다
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

# "weights" 폴더에 학습된 모델 "ECO_Lite_rgb_model_Kinetics.pth.tar"를 다운로드하여 배치하세요.
# https://github.com/mzolfaghari/ECO-pytorch 의
# https://drive.google.com/open?id=1XNIq7byciKgrn011jLBggd2g79jKX4uD

# Kinematics 동영상 데이터 세트의 DataLoader를 준비

In [3]:
from utils.kinetics400_eco_dataloader import make_datapath_list, VideoTransform, get_label_id_dictionary, VideoDataset

# vieo_list 작성
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)

# 전처리 설정
resize, crop_size = 224, 224
mean, std = [104, 117, 123], [1, 1, 1]
video_transform = VideoTransform(resize, crop_size, mean, std)

# 라벨 사전 작성
label_dicitionary_path = './video_download/kinetics_400_label_dicitionary.csv'
label_id_dict, id_label_dict = get_label_id_dictionary(label_dicitionary_path)

# Dataset 작성
# num_segments 는 동영상을 어떻게 분할해 사용할지를 결정
val_dataset = VideoDataset(video_list, label_id_dict, num_segments=16,
                           phase="val", transform=video_transform, img_tmpl='image_{:05d}.jpg')

# DataLoader로 합니다
batch_size = 8
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 동작 확인
batch_iterator = iter(val_dataloader)  # 반복자로 변환
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1번째 요소를 꺼낸다
print(imgs_transformeds.shape)


torch.Size([8, 16, 3, 224, 224])


# ECO 모델 구현

In [4]:
from utils.eco import ECO_2D, ECO_3D

class ECO_Lite(nn.Module):
    def __init__(self):
        super(ECO_Lite, self).__init__()

        # 2D Net 모듈
        self.eco_2d = ECO_2D()

        # 3D Net 모듈
        self.eco_3d = ECO_3D()

        # 클래스 분류의 전결합층
        self.fc_final = nn.Linear(in_features=512, out_features=400, bias=True)

    def forward(self, x):
        '''
        입력 x는 torch.Size([batch_num, num_segments=16, 3, 224, 224]))
        '''

        # 입력 x의 각 차원의 크기를 취득
        bs, ns, c, h, w = x.shape

        # x를 (bs*ns, c, h, w)로 크기를 변환한다
        out = x.view(-1, c, h, w)
        # (주석)
        # PyTorch의 Conv2D는 입력 크기가 (batch_num, c, h, w)만 허용되므로
        # (batch_num, num_segments, c, h, w)는 처리할 수 없다
        # 지금은 2차원 화상을 따로 처리하므로, num_segments는 batch_num의 차원에 넣어도 좋으므로
        # (batch_num×num_segments, c, h, w)로 크기를 변환한다

        # 2D Net 모듈 출력 torch.Size([batch_num×16, 96, 28, 28])
        out = self.eco_2d(out)

        # 2차원 화상을 텐서로 3차원용으로 변환
        # num_segments를 batch_num의 차원으로 넣은 것을 원래대로 되돌림
        out = out.view(-1, ns, 96, 28, 28)

        # 3D Net 모듈 출력 torch.Size([batch_num, 512])
        out = self.eco_3d(out)

        # 클래스 분류의 전결합층 출력 torch.Size([batch_num, class_num=400])
        out = self.fc_final(out)

        return out


In [5]:
net = ECO_Lite()
net

ECO_Lite(
  (eco_2d): ECO_2D(
    (basic_conv): BasicConv(
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1_relu_7x7): ReLU(inplace)
      (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3_reduce): ReLU(inplace)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3): ReLU(inplace)
      (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (inception_a): InceptionA(
      (inception_3a_1x1):

# 확습된 모델 로드

In [6]:
# 학습된 모델을 로드하는 함수 정의
def load_pretrained_ECO(model_dict, pretrained_model_dict):
    '''ECO 학습된 모델을 로드하는 함수
    이번에 구축한 ECO는 학습된 모델과 레이어 순서가 같지만 이름이 다름
    '''

    # 현재 네트워크 모델의 파라미터명
    param_names = []  # 파라미터명을 저장해 나간다
    for name, param in model_dict.items():
        param_names.append(name)

    # 현재 네트워크 정보를 복사하여 새로운 state_dict를 작성
    new_state_dict = model_dict.copy()

    # 새 state_dict에 학습된 값을 대입
    print("학습된 파라미터를 로드합니다")
    for index, (key_name, value) in enumerate(pretrained_model_dict.items()):
        name = param_names[index]  # 현재 네트워크에서 파라미터명을 취득
        new_state_dict[name] = value  # 값을 넣는다

        # 무엇에 로드된 것인지를 표시
        print(str(key_name)+"→"+str(name))

    return new_state_dict


# 학습된 모델을 로드
net_model_ECO = "./weights/ECO_Lite_rgb_model_Kinetics.pth.tar"
pretrained_model = torch.load(net_model_ECO, map_location='cpu')
pretrained_model_dict = pretrained_model['state_dict']
# (주석)
# pth가 tar로 압축되는 것은, state_dict 이외의 정보도 함께 저장되어 있기 때문이다.
# 따라서 읽어들일 때에는 사전형 변수로 되어 있으므로, ['state_dict']을 지정한다.

# 현재 모델의 변수명 등을 취득
model_dict = net.state_dict()

# 학습된 모델의 state_dict을 취득
new_state_dict = load_pretrained_ECO(model_dict, pretrained_model_dict)

# 학습된 모델의 파라미터를 대입
net.eval()  # ECO 네트워크를 추론 모드로
net.load_state_dict(new_state_dict)


학습된 파라미터를 로드합니다
module.base_model.conv1_7x7_s2.weight→eco_2d.basic_conv.conv1_7x7_s2.weight
module.base_model.conv1_7x7_s2.bias→eco_2d.basic_conv.conv1_7x7_s2.bias
module.base_model.conv1_7x7_s2_bn.weight→eco_2d.basic_conv.conv1_7x7_s2_bn.weight
module.base_model.conv1_7x7_s2_bn.bias→eco_2d.basic_conv.conv1_7x7_s2_bn.bias
module.base_model.conv1_7x7_s2_bn.running_mean→eco_2d.basic_conv.conv1_7x7_s2_bn.running_mean
module.base_model.conv1_7x7_s2_bn.running_var→eco_2d.basic_conv.conv1_7x7_s2_bn.running_var
module.base_model.conv1_7x7_s2_bn.num_batches_tracked→eco_2d.basic_conv.conv1_7x7_s2_bn.num_batches_tracked
module.base_model.conv2_3x3_reduce.weight→eco_2d.basic_conv.conv2_3x3_reduce.weight
module.base_model.conv2_3x3_reduce.bias→eco_2d.basic_conv.conv2_3x3_reduce.bias
module.base_model.conv2_3x3_reduce_bn.weight→eco_2d.basic_conv.conv2_3x3_reduce_bn.weight
module.base_model.conv2_3x3_reduce_bn.bias→eco_2d.basic_conv.conv2_3x3_reduce_bn.bias
module.base_model.conv2_3x3_reduce_bn.runn

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

# 추론(동영상 데이터의 클래스 분류)

In [7]:
# 추론
net.eval()  # ECO 네트워크를 추론 도ㅡ로

batch_iterator = iter(val_dataloader)  # 반복자로 변환
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1번째 요소를 꺼낸다

with torch.set_grad_enabled(False):
    outputs = net(imgs_transformeds)  # ECO로 추론

print(outputs.shape)  # 출력 크기


torch.Size([8, 400])


In [8]:
# 예측 결과 상위 5개를 표시합니다
def show_eco_inference_result(dir_path, outputs_input, id_label_dict, idx=0):
    '''미니 배치의 각 데이터에 대해, 추론 결과의 상위를 출력하는 함수 정의'''
    print("파일: ", dir_path[idx])  # 파일명

    outputs = outputs_input.clone()  # 사본 작성

    for i in range(5):
        '''1~5위까지 표시'''
        output = outputs[idx]
        _, pred = torch.max(output, dim=0)  # 확률 최대치 라벨을 예측
        class_idx = int(pred.numpy())  # 클래스 ID 출력
        print("예측 {}위: {}".format(i+1, id_label_dict[class_idx]))
        outputs[idx][class_idx] = -1000  # 최대치였던 것을 없앰(작게 한다)


# 예측 실시
idx = 0
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)


파일:  ./data/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028_000038
예측 1위: arm wrestling
예측 2위: headbutting
예측 3위: stretching leg
예측 4위: shaking hands
예측 5위: tai chi


In [9]:
# 예측 실시
idx = 4
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)


파일:  ./data/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012
예측 1위: bungee jumping
예측 2위: trapezing
예측 3위: abseiling
예측 4위: swinging on something
예측 5위: climbing a rope


끝